## Step 4: Analytical processing on denormalized data


In [ ]:
from flask import Flask, jsonify
from pymongo import MongoClient
import pandas as pd
import os
from dotenv import load_dotenv

app = Flask(__name__)


load_dotenv()

Client = MongoClient(os.getenv("MONGO_URI"))

db = Client[os.getenv("MONGO_DB_NAME")]

coll = db[os.getenv("MONGO_COLLECTION")]


def load_data():
    data = list(coll.find({}, {'_id': 0}))
    df = pd.json_normalize(data)


    if 'TotalPrice' in df.columns:
        df['Revenue'] = df['TotalPrice'] 
    elif 'Quantity' in df.columns and 'Product.Unitprice' in df.columns:
        df['Revenue'] = df['Quantity'] * df['Product.Unitprice']  
    else:
        df['Revenue'] = 0


    if 'Date' in df.columns:
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

    return df



@app.route('/data', methods=['GET'])
def get_data():
    df = load_data()
    return jsonify(df.to_dict(orient='records'))


@app.route('/ramadan_vs_other', methods=['GET'])
def ramadan_vs_other():
    df = load_data()
    ramadan_start = "2023-03-23"
    ramadan_end = "2023-04-21"

    df['InRamadan'] = df['Date'].between(ramadan_start, ramadan_end)
    result = {
        "Ramadan_Sales": float(df[df['InRamadan']]['Revenue'].sum()),
        "Other_Sales": float(df[~df['InRamadan']]['Revenue'].sum())
    }
    return jsonify(result)


@app.route('/topcustomer', methods=['GET'])
def top_customer():
    df = load_data()

    if 'Customer.Name' in df.columns and 'TotalPrice' in df.columns:
        top = (
            df.groupby('Customer.Name', as_index=False)['TotalPrice']
            .sum()
            .sort_values(by='TotalPrice', ascending=False)
            .head(10)
        )
        return jsonify(top.to_dict(orient='records'))
    else:
        return jsonify({"error": "Missing columns in data"})


@app.route("/branchrevenue", methods=["GET"])
def branch_revenue():
    try:
        df = load_data()
        branch_rev = (
            df.groupby("Branch.BranchName")["Revenue"]
            .sum()
            .reset_index()
            .sort_values(by="Revenue", ascending=False)
        )
        return jsonify(branch_rev.to_dict(orient="records"))
    except Exception as e:
        return jsonify({"error": str(e)})


@app.route("/dailysales", methods=["GET"])
def daily_sales():
    try:
        df = load_data()
        daily_sales_df = (
            df.groupby("Date")["Revenue"]
            .sum()
            .reset_index()
            .sort_values(by="Revenue", ascending=False)
        )
        return jsonify(daily_sales_df.to_dict(orient="records"))
    except Exception as e:
        return jsonify({"error": str(e)})

@app.route("/quartersales", methods=["GET"])
def quarter_sales():
    try:
        df = load_data()
        df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
        df["Quarter"] = df["Date"].dt.to_period("Q")

        last_quarter = df["Quarter"].max()
        branch_quarter_sales = (
            df[df["Quarter"] == last_quarter]
            .groupby("Branch.BranchName")["Revenue"]
            .sum()
            .sort_values(ascending=False)
            .reset_index()
        )

        return jsonify(branch_quarter_sales.to_dict(orient="records"))
    except Exception as e:
        return jsonify({"error": str(e)})

@app.route("/topproduct", methods=["GET"])
def topproduct():
    df = load_data()

    
    if "Product.ProductName" in df.columns and "Product.Unitprice" in df.columns and "Quantity" in df.columns:

       
        df["ProductRevenue"] = df["Product.Unitprice"] * df["Quantity"]

        
        top_products = (
            df.groupby("Product.ProductName", as_index=False)["ProductRevenue"]
            .sum()
            .sort_values(by="ProductRevenue", ascending=False)
            .head(10)
        )

        return jsonify(top_products.to_dict(orient="records"))
    else:
        return jsonify({"error": "Missing columns in data"})
@app.route("/monthlysales", methods=["GET"])
def monthly_sales():
    try:
        df = load_data()

    
        df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

        df["Month"] = df["Date"].dt.to_period("M").astype(str)

        monthly_df = (
            df.groupby("Month")["Revenue"]
            .sum()
            .reset_index()
            .sort_values(by="Month")
        )

        return jsonify(monthly_df.to_dict(orient="records"))

    except Exception as e:
        return jsonify({"error": str(e)})



In [ ]:
import threading
import time
def run_flask():
    app.run(port=5000,debug=False,use_reloader=False)
threading.Thread(target=run_flask).start()
time.sleep(1)    

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


127.0.0.1 - - [18/Nov/2025 09:28:18] "GET /topcustomer HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2025 09:31:01] "GET /quartersales HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2025 09:31:31] "GET /dailysales HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2025 09:36:26] "GET /topproduct HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2025 09:38:03] "GET /monthlysales HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2025 09:53:19] "GET /monthlysales HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2025 09:55:48] "GET /topproduct HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2025 09:56:03] "GET /monthlysales HTTP/1.1" 200 -


In [3]:
data = list(coll.find({}, {'_id': 0}))
df = pd.json_normalize(data)
print(df.columns)

Index(['SaleID', 'Date', 'Quantity', 'TotalPrice', 'Customer.CustomerID',
       'Customer.Name', 'Customer.Gender', 'Customer.City', 'Branch.BranchID',
       'Branch.BranchName', 'Branch.City', 'Product.ProductID',
       'Product.ProductName', 'Product.Category', 'Product.Unitprice'],
      dtype='object')
